In [1]:
# 필요한 모듈과 라이브러리를 로딩
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, os, math, random, sys, datetime
from datetime import date, timedelta, datetime
import pyautogui
from tqdm import tqdm

# chromedriver 자동 설치
import chromedriver_autoinstaller

from selenium.webdriver.common.action_chains import ActionChains


In [2]:
# 작업 시간과 고유 dir 등 생성
n = time.localtime()
s = '%04d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday)
s_time = time.time( )


a = datetime.now()-timedelta(days=1)
a = a.strftime('%Y-%m-%d')
b = datetime.now()-timedelta(days=2)
b = b.strftime('%Y-%m-%d')
print(b, a)

2021-05-18 2021-05-19


In [3]:
print("=" *80)
print("         OPINION 크롤러입니다.")
print("=" *80)

f_dir = os.path.expanduser('~') + f'\\Desktop\\daily\\{b}\\' # 운영체제 상관없이 바탕화면 'crawler-bot' 폴더에 작업

print("=" *80)
print("데이터 크롤링을 시작합니다.")
print("=" *80)

         OPINION 크롤러입니다.
데이터 크롤링을 시작합니다.


In [4]:
# 저장될 파일 경로와 이름을 지정
# 파일을 저장할 폴더 위치를 만들고, 중복일 경우 해당 폴더를 삭제하고 새로 생성합니다.
try : 
    os.makedirs(f_dir)
except : pass
else : 
    os.chdir(f_dir)

fc_name=f_dir+'\\'+f'ddanzi.csv'

In [5]:
# 크롤링 데이터가 들어갈 리스트 생성
domain2 = []    # 크롤링 도메인
keyword2 = []   # 크롤링 키워드
title2 = []     # 게시물 제목
user2 = []      # 게시물 작성자
url2 = []       # 게시물 url
time2 = []      # 게시물 작성 시간
view_cnt2 = []  # 게시물 뷰 카운트
like_cnt2 = []  # 추천/공감 카운트
text2 = []      # 게시물 내용
com_cnt2 = []   # 댓글 수
com_user2 = []  # 작성자 본인을 제외한 댓글 유저
com_text2 = []  # 댓글 user_id & text

In [6]:
# chromedriver-autoinstaller 이용
chromedriver_autoinstaller.install(cwd=True)

# HEADLESS MODE
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('window-size=1920x1080') 
# options.add_argument("--disable-gpu")
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
args = ["hide_console", ]

# 알림창 끄기
options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

# 웹사이트 접속 후 해당 메뉴로 이동
driver = webdriver.Chrome(options=options,service_args=args)
driver.maximize_window()

In [7]:
# 1. 딴지 크롤링
query_url= ('https://www.ddanzi.com/free')

driver.get(query_url)
driver.implicitly_wait(5)

In [8]:
domain1 = "딴지"
print("==========",domain1,"크롤링 진행 중 ==========")

========== 딴지 크롤링 진행 중 ==========


In [9]:
def login() :
    


    tmp_id = "ddive1041"
    tmp_pw = "q1w2e3r4!@"

    driver.find_element_by_link_text('로그인').click()
    time.sleep(1)
    # id 입력
    driver.find_element_by_xpath('//*[@id="uid"]').click()
    for i in tmp_id :
        time.sleep(0.1)
        driver.find_element_by_xpath('//*[@id="uid"]').send_keys(i)

    # pw 입력    
    driver.find_element_by_xpath('//*[@id="upw"]').click()
    for j in tmp_pw :
        time.sleep(0.1)
        driver.find_element_by_xpath('//*[@id="upw"]').send_keys(j)

    driver.find_element_by_xpath('//*[@id="regularsubmit"]/td/button').click()
    time.sleep(1)

In [10]:
login()

In [11]:
# # 인증글 모아보기 클릭

# driver.find_element_by_link_text("인증글 모아보기").click()
# time.sleep(3)

In [12]:
page_num = 40
while_code = 0
while True :
    if while_code == 1 :
        print("\n\n")
        print("===== 1차 게시물 수집 작업 완료...")
        print("===== 2차 게시물 내용 수집으로 넘어갑니다.")
        print("\n\n")
        break
    

        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    item_box = soup.find_all(name='tbody')[-1].find_all('tr')              
        
    
    post_num = 0
    
    for item in item_box :
        
        try : # 공지글만 class가 있음, 공지글 제외
            item['class']
            post_num += 1
            cut_post_num = post_num
        except :
            post_num += 1
            page_num += 1

            # 크롤링 시작
            post_time = item.find(name='td', attrs={'class':'time'}).text        


            # 시간만 표시되는 오늘 게시물이거나, 올해 게시물만 크롤링
            if post_time > a or (datetime.now().strftime('%Y-%m-%d') != a and ":" in post_time):
                time_code=0
            elif datetime.now().strftime('%Y-%m-%d') == a and ":" in post_time :
                time_code = 1 
            elif post_time <= a and post_time >= b  :
                time_code = 1
            else : # 시간 영역이 맞지 않는 경우 time_code=2 로 변경
                time_code = 2
            
            print("time_scout:",post_time, time_code)
            if time_code == 1 :

                print("크롤링 도메인 : ",domain1)

                title1 = item.find(name='td', attrs={'class':'title'}).find('a').text.strip()
                user1 = item.find(name='td', attrs={'class':'author txtleft'}).text.strip()
                url1 = item.find(name='td', attrs={'class':'title'}).find('a')['href']
                time1 = item.find(name='td', attrs={'class':'time'}).text
                view_cnt1 = item.find(name='td', attrs={'class':'readNum'}).text
                like_cnt1 = item.find(name='td', attrs={'class':'voteNum'}).text.strip()
                
                try :
                    com_cnt1 = item.find(name='td', attrs={'class':'title'}).find(name='span', attrs={'class':'talk'}).text.replace("[","").replace("]","").strip()
                except :
                    com_cnt1 = 0

                print("게시물 제목 :", title1)
                print("작성자 :", user1)
                print("게시물 url :", url1)
                print("업로드 시간 :", time1)
                print("조회수 :", view_cnt1)
                print("추천수 :", like_cnt1)
                print("댓글수 :", com_cnt1)
                print("\n")

                domain2.append(domain1)
#                 keyword2.append(keyword)
                title2.append(title1)
                user2.append(user1)
                url2.append(url1)
                time2.append(time1)
                view_cnt2.append(view_cnt1)
                like_cnt2.append(like_cnt1)
                com_cnt2.append(com_cnt1)    


            elif time_code == 2 :
                while_code = 1
                break
                

    
    if post_num == cut_post_num : # 게시물이 나오지 않고 공지만 있는 경우 : 계속 검색으로 우회
        print("=====   계속 검색을 시도합니다.   =====")
        driver.find_element_by_link_text("계속 검색").click()
        driver.implicitly_wait(3)
        time.sleep(2)
        page_num = 1
        
    else :
        print("=====   다음 페이지로 넘어갑니다.   =====")
        try :
            
            cur_url = driver.current_url        
            cur_page = int(cur_url.split("&page=")[1])
            print("scout 03 :", cur_url)
            next_page = cur_page + 1
            next_url = cur_url.split("&page=")[0] + "&page=" + str(next_page)
            print("scout 04 :", cur_url.split("&page="))
            print("scout 05 :", next_url)
            driver.get(next_url)
            driver.implicitly_wait(3)
            time.sleep(3)
            
        except :
            try :                
                print("scout 01:", driver.current_url )
                driver.find_element_by_link_text("2").click()
                driver.implicitly_wait(3)
                time.sleep(2)
                print("scout 02:", driver.current_url )
            except : # 페이지가 하나밖에 표시되지 않은 경우 - 계속 검색으로 우회
                print("=====   계속 검색을 시도합니다.   =====")
                driver.find_element_by_link_text("계속 검색").click()
                driver.implicitly_wait(3)
                time.sleep(2)
                page_num = 1
                
                

time_scout: 15:03:35 0
time_scout: 15:03:15 0
time_scout: 15:03:08 0
time_scout: 15:01:10 0
time_scout: 15:00:17 0
time_scout: 14:59:52 0
time_scout: 14:59:33 0
time_scout: 14:59:11 0
time_scout: 14:58:58 0
time_scout: 14:58:53 0
time_scout: 14:58:34 0
time_scout: 14:58:33 0
time_scout: 14:57:24 0
time_scout: 14:56:56 0
time_scout: 14:56:29 0
time_scout: 14:56:06 0
time_scout: 14:54:29 0
time_scout: 14:54:23 0
time_scout: 14:54:22 0
time_scout: 14:54:07 0
time_scout: 14:53:47 0
time_scout: 14:53:04 0
time_scout: 14:52:59 0
time_scout: 14:52:41 0
time_scout: 14:52:26 0
time_scout: 14:52:04 0
time_scout: 14:51:22 0
time_scout: 14:51:14 0
time_scout: 14:50:30 0
time_scout: 14:50:14 0
=====   다음 페이지로 넘어갑니다.   =====
scout 01: https://www.ddanzi.com/free
scout 02: https://www.ddanzi.com/index.php?mid=free&page=2
time_scout: 14:49:43 0
time_scout: 14:49:20 0
time_scout: 14:48:52 0
time_scout: 14:47:24 0
time_scout: 14:46:59 0
time_scout: 14:46:27 0
time_scout: 14:46:24 0
time_scout: 14:46:06 

time_scout: 12:21:10 0
time_scout: 12:21:09 0
time_scout: 12:20:38 0
time_scout: 12:19:25 0
time_scout: 12:18:54 0
time_scout: 12:18:28 0
time_scout: 12:18:05 0
time_scout: 12:17:46 0
time_scout: 12:17:23 0
time_scout: 12:17:14 0
time_scout: 12:16:45 0
time_scout: 12:15:23 0
time_scout: 12:14:26 0
time_scout: 12:13:36 0
time_scout: 12:13:28 0
time_scout: 12:13:17 0
time_scout: 12:13:02 0
time_scout: 12:12:46 0
time_scout: 12:12:42 0
time_scout: 12:11:20 0
time_scout: 12:11:04 0
time_scout: 12:11:01 0
time_scout: 12:10:37 0
time_scout: 12:10:30 0
time_scout: 12:10:13 0
time_scout: 12:10:07 0
time_scout: 12:09:18 0
time_scout: 12:08:14 0
time_scout: 12:08:08 0
time_scout: 12:07:52 0
=====   다음 페이지로 넘어갑니다.   =====
scout 03 : https://www.ddanzi.com/index.php?mid=free&page=11
scout 04 : ['https://www.ddanzi.com/index.php?mid=free', '11']
scout 05 : https://www.ddanzi.com/index.php?mid=free&page=12
time_scout: 12:07:16 0
time_scout: 12:06:13 0
time_scout: 12:05:44 0
time_scout: 12:05:41 0
ti

time_scout: 10:01:08 0
time_scout: 10:01:04 0
time_scout: 09:59:45 0
time_scout: 09:59:41 0
time_scout: 09:59:41 0
time_scout: 09:59:34 0
time_scout: 09:59:22 0
time_scout: 09:59:17 0
time_scout: 09:59:13 0
time_scout: 09:59:05 0
time_scout: 09:57:22 0
time_scout: 09:56:52 0
time_scout: 09:54:47 0
time_scout: 09:54:22 0
time_scout: 09:54:02 0
time_scout: 09:53:42 0
time_scout: 09:53:41 0
time_scout: 09:53:00 0
time_scout: 09:52:47 0
time_scout: 09:52:42 0
time_scout: 09:51:31 0
time_scout: 09:51:29 0
time_scout: 09:51:16 0
time_scout: 09:51:06 0
time_scout: 09:50:41 0
time_scout: 09:50:31 0
time_scout: 09:49:39 0
time_scout: 09:47:57 0
time_scout: 09:47:47 0
time_scout: 09:47:30 0
=====   다음 페이지로 넘어갑니다.   =====
scout 03 : https://www.ddanzi.com/index.php?mid=free&page=21
scout 04 : ['https://www.ddanzi.com/index.php?mid=free', '21']
scout 05 : https://www.ddanzi.com/index.php?mid=free&page=22
time_scout: 09:47:29 0
time_scout: 09:47:26 0
time_scout: 09:47:20 0
time_scout: 09:47:10 0
ti

time_scout: 01:28:17 0
time_scout: 01:26:51 0
time_scout: 01:25:42 0
time_scout: 01:23:43 0
time_scout: 01:19:05 0
time_scout: 01:16:48 0
time_scout: 01:13:15 0
time_scout: 01:12:59 0
time_scout: 01:12:44 0
time_scout: 01:11:27 0
time_scout: 01:08:57 0
time_scout: 01:08:06 0
time_scout: 01:06:28 0
time_scout: 01:06:17 0
time_scout: 01:04:21 0
time_scout: 01:03:04 0
time_scout: 01:02:50 0
time_scout: 01:02:40 0
time_scout: 01:02:16 0
time_scout: 01:02:01 0
time_scout: 01:01:19 0
time_scout: 00:59:46 0
time_scout: 00:58:17 0
time_scout: 00:57:39 0
time_scout: 00:57:18 0
time_scout: 00:56:25 0
time_scout: 00:55:50 0
time_scout: 00:54:46 0
time_scout: 00:54:27 0
time_scout: 00:54:25 0
=====   다음 페이지로 넘어갑니다.   =====
scout 03 : https://www.ddanzi.com/index.php?mid=free&page=31
scout 04 : ['https://www.ddanzi.com/index.php?mid=free', '31']
scout 05 : https://www.ddanzi.com/index.php?mid=free&page=32
time_scout: 00:53:41 0
time_scout: 00:53:24 0
time_scout: 00:52:28 0
time_scout: 00:49:27 0
ti

time_scout: 2021-05-19 1
크롤링 도메인 :  딴지
게시물 제목 : 전기차 전기바이크가 빨리 대중화되었음 합니다.
작성자 : 인생역전?인생여전!ㅜㅜ
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=36&document_srl=682183156
업로드 시간 : 2021-05-19
조회수 : 473
추천수 : 1
댓글수 : 13


time_scout: 2021-05-19 1
크롤링 도메인 :  딴지
게시물 제목 : 국회의원 뽑아줬어요, 이젠 어떻게 해야 하죠?
작성자 : 라면만박스
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=36&document_srl=682183092
업로드 시간 : 2021-05-19
조회수 : 207
추천수 : 3
댓글수 : 2


time_scout: 2021-05-19 1
크롤링 도메인 :  딴지
게시물 제목 : 주식의 가격 심리..
작성자 : 수습은안한다
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=36&document_srl=682183087
업로드 시간 : 2021-05-19
조회수 : 601
추천수 : 2
댓글수 : 5


time_scout: 2021-05-19 1
크롤링 도메인 :  딴지
게시물 제목 : 딴지도 하면서 타 대형 커뮤들도 꾸준히 가야 하는 eu
작성자 : 반봄맬
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=36&document_srl=682183053
업로드 시간 : 2021-05-19
조회수 : 1883
추천수 : 22
댓글수 : 7


time_scout: 2021-05-19 1
크롤링 도메인 :  딴지
게시물 제목 : 코인 하락의 의심점
작성자 : 짤짤이인생
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=36&d

...

time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 국짐당은 도토리들만 있다 ㅋㅋㅋ
작성자 : 귀신아범
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=148&document_srl=681970544
업로드 시간 : 2021-05-18
조회수 : 205
추천수 : 2
댓글수 : 0


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 만고불변의 페미들 공통점……
작성자 : generation
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=148&document_srl=681970538
업로드 시간 : 2021-05-18
조회수 : 323
추천수 : 
댓글수 : 1


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 방송에서 '독고다이'...😩
작성자 : 🌝Der_Mond
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=148&document_srl=681970344
업로드 시간 : 2021-05-18
조회수 : 700
추천수 : 5
댓글수 : 3


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 현재 야구 순위
작성자 : 雨天☔️JS♥️YK
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=148&document_srl=681970319
업로드 시간 : 2021-05-18
조회수 : 763
추천수 : 
댓글수 : 9


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 정청래 째려보는 하태경 ㅋㅋㅋ
작성자 : 이란왕족
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=148&document_srl=681970255
업로

time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 포털 사이트에 기사 한개도 안보인다
작성자 : 버꾸랑
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=150&document_srl=681966332
업로드 시간 : 2021-05-18
조회수 : 437
추천수 : 6
댓글수 : 1


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 넷플릭스 우먼 인 윈도우 짧은 평
작성자 : director'scut
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=150&document_srl=681966289
업로드 시간 : 2021-05-18
조회수 : 963
추천수 : 2
댓글수 : 6


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 오세훈 의자가 대단하내
작성자 : 포도사세요
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=150&document_srl=681966269
업로드 시간 : 2021-05-18
조회수 : 979
추천수 : 
댓글수 : 1


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 김용민 수석최고, "철저한 진상규명과 함께 5.18정신을 기리는 데 앞장서겠습니다."
작성자 : 🌝Der_Mond
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=150&document_srl=681966254
업로드 시간 : 2021-05-18
조회수 : 347
추천수 : 17
댓글수 : 2


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 5.18을 폭동이라 생각하는 놈들이 이렇게 많다니...
작성자 : 눈먼새
게시물 url : https://www.ddanzi.com/

time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 한국방송 한번 출연하고 인생이 바뀐 외국인
작성자 : moonduck
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=152&document_srl=681962501
업로드 시간 : 2021-05-18
조회수 : 13020
추천수 : 11
댓글수 : 7


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : Dodie - "Come Together" cover ..
작성자 : simpleisbest
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=152&document_srl=681962499
업로드 시간 : 2021-05-18
조회수 : 82
추천수 : 
댓글수 : 0


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 자동차 체납액을 전체 확인하려면?
작성자 : bros.b
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=152&document_srl=681962487
업로드 시간 : 2021-05-18
조회수 : 365
추천수 : 
댓글수 : 4


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 박시영TV 출연하신 최강욱 당대표님 말씀이....
작성자 : 초코🐾촵촵
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=152&document_srl=681960262
업로드 시간 : 2021-05-18
조회수 : 8189
추천수 : 107
댓글수 : 7


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 퇴근까지 3시간 남았네혀~~
작성자 : Time_is_ing
게시물 url : https://www.ddanzi.com

time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 이번 아이패드 구매 견적 대충 내봤읍니다 ㄷㄷ
작성자 : Time_is_ing
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=154&document_srl=681958131
업로드 시간 : 2021-05-18
조회수 : 717
추천수 : 
댓글수 : 15


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 일산화탄소 센서 소자... 신뢰성 높은 회사가 어딜까요?
작성자 : IDm00n
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=154&document_srl=681958109
업로드 시간 : 2021-05-18
조회수 : 231
추천수 : 
댓글수 : 1


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 근데, 어차피 엄중이랑 송영길이 똥싸면 문재인 대통령께서 다 욕먹지 않나요?
작성자 : 권성수주심,임정엽재판장,김선희
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=154&document_srl=681958057
업로드 시간 : 2021-05-18
조회수 : 222
추천수 : 
댓글수 : 2


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 나이들어 공부할때는  집중력 어찌 높이나요
작성자 : 쿨잠
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=154&document_srl=681957786
업로드 시간 : 2021-05-18
조회수 : 2156
추천수 : 7
댓글수 : 14


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 5.18 전야제 재밌었습니다! :) (욕 주의;)
작성자 : 눈팅만하던총각
게시

time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 유행했다가 급 시들시들해진 빵
작성자 : 푸른무지개🌈😼😸😺🐶🏌
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=156&document_srl=681953516
업로드 시간 : 2021-05-18
조회수 : 7416
추천수 : 7
댓글수 : 18


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : Giulio Cesare : Son nata a lagrimar 有
작성자 : ▶◀YULLS
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=156&document_srl=681953512
업로드 시간 : 2021-05-18
조회수 : 76
추천수 : 3
댓글수 : 2


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 보안프로그램 왜 안없애지
작성자 : 甲
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=156&document_srl=681953259
업로드 시간 : 2021-05-18
조회수 : 323
추천수 : 2
댓글수 : 2


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 이번 팔레스타인 공습 붕괴 장면
작성자 : 뻘글러17호
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=156&document_srl=681953367
업로드 시간 : 2021-05-18
조회수 : 664
추천수 : 1
댓글수 : 1


time_scout: 2021-05-18 1
크롤링 도메인 :  딴지
게시물 제목 : 대만 이러다가 TSMC에서 확진자 나오면
작성자 : 피프카시
게시물 url : https://www.ddanzi.com/index.php?mid=free&page=

In [16]:
def crawling02() :
    global batch_cnt, while_code

    print(f"======   키워드 게시물 총 {len(url2)}개입니다.   =====")
    print(f"=====   총 {math.ceil(len(url2) / 10)} 개의 batch로 나눠 진행합니다.   ======")

    while True :
        print(f"현재 {batch_cnt+1}번째 batch 진행 중...")
        driver.refresh()

        try :
            login()
        except :
            pass

        time.sleep(1)

        url_list = [x for x in url2[batch_cnt*10:batch_cnt*10+10]] # 10개씩 끊기

        batch_user2 = []
        batch_text2 = []
        batch_com_text2 = []

        for url in tqdm(url_list) :
            print("url :", url)

            com_cnt = 0

            while True :

                com_cnt += 1

                post_code = 0


                try :

                    driver.get(url)
                    driver.implicitly_wait(3)
                    time.sleep(random.randint(1,3))

                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')

                    text1 = soup.find(name='div', attrs={'class':'read_content'}).text.strip().replace("\n", "")
                    com_user_list = soup.find(name='div', attrs={'id':'cmt_list'}).find_all(name='div', attrs={'class':'fbItem_left'})
                    com_text_list = soup.find(name='div', attrs={'id':'cmt_list'}).find_all(name='div', attrs={'class':'fbItem_right'})
                    tmp_list = []
                    user_list = []

                    for k in range(len(com_user_list)) :

                        string = "작성자 : " + com_user_list[k].text.strip().replace("\n","").replace("\t", "") + " :: " + com_text_list[k].text.strip().replace("\n","").replace("\t", "")
                        com_list.append(string)
                        user_list.append(com_user_list[k].find(name='h4',attrs={'class':'author'}).text.strip().replace("\n","").replace("\t", "").replace("본문 글쓴이",""))

                    for k in range(len(com_user_list)) :
                #         print(com_user_list[k].text.strip().replace("\n",""),com_text_list[k].text.strip().replace("\n",""))
                        string = com_user_list[k].text.strip().replace("\n","").replace("\t", "") + " : " + com_text_list[k].text.strip().replace("\n","").replace("\t", "")
                        tmp_list.append(string)
                    com_text1 = tmp_list


                    print("본문 :", text1)
                    print("댓글 유저 :", user_list)
                    print("댓글 :", com_text1)
                    print("\n")

                    batch_text2.append(text1)
                    batch_user2.append(user_list)
                    batch_com_text2.append(com_text1)

                    post_code = 1

                    driver.refresh()

                    break

                except : # 글이 삭제된 경우

                    except_cnt = 0

                    while except_cnt <= 2 :
                        except_code = 0

                        try :
                            except_cnt += 1

                            driver.refresh()
                            driver.get(url)
                            driver.implicitly_wait(3)
                            time.sleep(1)

                            html = driver.page_source
                            soup = BeautifulSoup(html, 'html.parser')

                            text1 = soup.find(name='div', attrs={'class':'read_content'}).text.strip().replace("\n", "")
                            com_user_list = soup.find(name='div', attrs={'id':'cmt_list'}).find_all(name='div', attrs={'class':'fbItem_left'})
                            com_text_list = soup.find(name='div', attrs={'id':'cmt_list'}).find_all(name='div', attrs={'class':'fbItem_right'})
                            user_list = []
                            com_list = []
    #                         tmp_list = []

                            for k in range(len(com_user_list)) :

                                string = "작성자 : " + com_user_list[k].text.strip().replace("\n","").replace("\t", "").replace("본문 글쓴이","") + " :: " + com_text_list[k].text.strip().replace("\n","").replace("\t", "")
                                com_list.append(string)
                                user_list.append(com_user_list[k].find(name='h4',attrs={'class':'author'}).text.strip().replace("\n","").replace("\t", "").replace("본문 글쓴이",""))

    #                         for k in range(len(com_user_list)) :
    #                     #         print(com_user_list[k].text.strip().replace("\n",""),com_text_list[k].text.strip().replace("\n",""))
    #                             string = com_user_list[k].text.strip().replace("\n","").replace("\t", "") + " : " + com_text_list[k].text.strip().replace("\n","").replace("\t", "")
    #                             tmp_list.append(string)
                            com_text1 = com_list


                            print("본문 :", text1)
                            print("댓글 유저 :", user_list)
                            print("댓글 :", com_list)
                            print("\n")

                            batch_text2.append(text1)
                            batch_user2.append(user_list)
                            batch_com_text2.append(com_list)

                            except_code = 1

                            driver.refresh()

                            break

                        except :
                            pass

                if except_code == 0 :
                    driver.refresh()

                    try :
                        login()
                    except :
                        pass

                    print("삭제된 게시물 입니다.")

                    try :                    
                        print("본문 :", text1)
                        print("댓글 유저:", user_list)
                        print("댓글 :", com_text1)
                        print("\n")                
                    except :
                        pass

                    text1 = "삭제된 게시물"
                    user_list = [""]
                    com_text1 = "삭제된 게시물"


                    print("본문 :", text1)
                    print("댓글 유저:", user_list)
                    print("댓글 :", com_text1)
                    print("\n")

                    batch_text2.append(text1)
                    batch_user2.append(user_list)
                    batch_com_text2.append(com_text1)

                    driver.refresh()

                    break

        for i in range(len(batch_text2)) :
            text2.append(batch_text2[i])
            com_text2.append(batch_com_text2[i])
            com_user2.append(batch_user2[i])
        driver.refresh()

        if batch_cnt*10+10 >= len(url2) :
            while_code = 1
            break

        batch_cnt += 1

    print("모든 크롤링 작업을 완료하였습니다.")



In [25]:
# batch_cnt = 0
# batch_cnt += 1
# login()
while_code = 0
while True :
    if while_code == 1 :
        break
        
    try :
        crawling02()
    except :
        driver.delete_all_cookies()
        time.sleep(1)
        driver.refresh()
        login()

======   키워드 게시물 총 3670개입니다.   =====
=====   총 367 개의 batch로 나눠 진행합니다.   ======
현재 147번째 batch 진행 중...


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=9709): Max retries exceeded with url: /session/d619ab6ccf4351c0aac61811ef84e533/cookie (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001400A2C21C0>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

In [26]:
print("Let's Test")

Let's Test


In [27]:
df = pd.DataFrame()

df['domain'] = pd.Series(domain2)
# df['keyword'] = pd.Series(keyword2)
df['title'] = pd.Series(title2)
df['user_name'] = pd.Series(user2)
df['url'] = pd.Series(url2)
df['date'] = pd.Series(time2)
df['view_cnt'] = pd.Series(view_cnt2)
df['like_cnt'] = pd.Series(like_cnt2)
df['user_text'] = pd.Series(text2)
df['com_cnt'] = pd.Series(com_cnt2)
df['com_user'] = pd.Series(com_user2)
df['com_text'] = pd.Series(com_text2)

df

,domain,title,writer,url,time,view_cnt,like_cnt,content,com_cnt,com_user,com_text
0,딴지,빌게이츠 이혼 사유가 불륜이라니,압도적일당,https://www.ddanzi.com/index.php?mid=free&page...,2021-05-19,7416,4,인류를 위한 자선 사업가가 불륜이라니 아이러니 하네요 ㄷㄷㄷ,29,"[이웃삼촌, 펭러뷰, 백서, N●천상천하You와독존, 싹틔운현미밥=인삼향, 세례자요...",[작성자 : 이웃삼촌 00:01:42 :: 부자는 뭐 다를까요. 다들 오욕칠정 덩어...
1,딴지,직급은 안올려주고 월급만올러주는건 ㅋㅋ,두둠칫뿡,https://www.ddanzi.com/index.php?mid=free&page...,2021-05-19,471,,인류를 위한 자선 사업가가 불륜이라니 아이러니 하네요 ㄷㄷㄷ,3,"[이웃삼촌, 펭러뷰, 백서, N●천상천하You와독존, 싹틔운현미밥=인삼향, 세례자요...",[이웃삼촌 00:01:42 : 부자는 뭐 다를까요. 다들 오욕칠정 덩어리 사람인데....
2,딴지,갑자기 생각나서 본게방 1회 찾아 듣는 중인데...😎ㅋ,🌝Der_Mond,https://www.ddanzi.com/index.php?mid=free&page...,2021-05-19,112,,얇고 길게ㅡ가라는 뜻이라 받아드리겟습니다 ㅋ,3,"[텐디쟁이, 두둠칫뿡, 빅퍼스]",[텐디쟁이 2021-05-19 23:59 : 월급은 안올려주고 직급만 올려주는것보단...
3,딴지,분노의 질주 이전편 하나도 안보고 이번에 개봉한거 처음봤는데요...,⬆️OneWay,https://www.ddanzi.com/index.php?mid=free&page...,2021-05-19,380,,추억이 새록새록 떠오르네요...😎ㅋㅋㅋㅋ https://youtu.be/B2j7VS...,15,"[▶◀생선🐱, 🌝Der_Mond, Alex_Yoda]","[▶◀생선🐱 00:01:43 : 소소하니 잼났는데 ㅎㅎ, 🌝Der_Mond본문 글쓴..."
4,딴지,자러가기 전에 후방,moonduck,https://www.ddanzi.com/index.php?mid=free&page...,2021-05-19,884,5,"이 영화 원래 이렇게 스토리의 계연성이나 과학적 합리성, 인물간의 상호관계 이런거 ...",4,"[불루스까이, ⬆️OneWay, 피자N우유, ⬆️OneWay, 피자N우유, [N🔴]...",[불루스까이 00:00:19 : 이전거 다보고 이번편만 아직 못봤는데..동네 건달들...
...,...,...,...,...,...,...,...,...,...,...,...
3665,딴지,애플워치 24시간 사용소감,director'scut,https://www.ddanzi.com/index.php?mid=free&page...,2021-05-18,796,1,NaN,6,NaN,NaN
3666,딴지,유로뉴스에 South Korea가...,고기돌이,https://www.ddanzi.com/index.php?mid=free&page...,2021-05-18,805,,NaN,4,NaN,NaN
3667,딴지,넷플) 더 테이블...,[N🔴]NDIDI™,https://www.ddanzi.com/index.php?mid=free&page...,2021-05-18,308,,NaN,3,NaN,NaN
3668,딴지,국짐의 5.18 행사들을 보며...,Jennie,https://www.ddanzi.com/index.php?mid=free&page...,2021-05-18,298,10,NaN,2,NaN,NaN


In [28]:
# csv 형태로 저장하기
df.to_csv(fc_name,encoding="utf-8-sig",index=False)

e_time = time.time( )
t_time = e_time - s_time

print("\n")
print("=" *80)
print("1.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("2.파일 저장 완료: csv 파일명 : %s " %fc_name)

driver.quit()
sys.exit("프로그램을 종료합니다.")



1.총 소요시간은 70124.7 초 입니다 
2.파일 저장 완료: csv 파일명 : C:\Users\JPARK-02\Desktop\daily\2021-05-18\\ddanzi.csv 


SystemExit: 프로그램을 종료합니다.

c:\python39\lib\site-packages\IPython\core\interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
